In [ ]:
import tensorflow
import matplotlib.pyplot as plt
import sklearn
import pandas
import numpy
import json
import os
import PIL
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from datetime import datetime
from sklearn.model_selection import train_test_split


De gebruikte afbeeldingen zijn 128x128 met 3 kleur kanalen (RGB)

In [ ]:
image_width = 128
image_height = 128
image_channels = 3

In [ ]:
batch_size = 2
epochs = 10

In [ ]:
model = tensorflow.keras.Sequential()

In [ ]:
model.add(Rescaling(1./255, input_shape=(image_height, image_width, 3)))

In [ ]:
model.add(Conv2D(32, (3,3), input_shape = (image_width, image_height, image_channels)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

In [ ]:
model.add(Conv2D(32, (3,3), input_shape = (image_width, image_height, image_channels)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

In [ ]:
model.add(Flatten())
model.add(Dense(64))

In [ ]:
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

Een custom callback om inzichtelijk te maken hoe het trainen werkt

In [ ]:
class CustomCallback(tensorflow.keras.callbacks.Callback):
    
    def on_train_begin(self, logs = None):
        print("Starting training")
        
    def on_train_end(self, logs = None):
        print("Stopped training")
        
    def on_epoch_begin(self, epoch, logs = None):
        print("Epoch begin")
        
    def on_epoch_end(self, epoch, logs = None):
        print("Epoch end")
    
    def on_test_begin(self, logs = None):
        print("Test start")
        
    def on_test_end(self, logs = None):
        print("Test end")
        
    def on_predict_begin(self, logs = None):
        print("Predicting")
        
    def on_predict_end(self, logs = None):
        print("End predicting")
    
    def on_train_batch_begin(self, batch, logs=None):
        print("Train batch begin")

    def on_train_batch_end(self, batch, logs=None):
        print("Train batch end")

    def on_test_batch_begin(self, batch, logs=None):
        print("Test batch begin")

    def on_test_batch_end(self, batch, logs=None):
        print("Test batch end")

    def on_predict_batch_begin(self, batch, logs=None):
        print("Predict batch begin")

    def on_predict_batch_end(self, batch, logs=None):
        print("Predict batch end")

De datasets worden uit de train- en validatie mappen gehaald. Hierbij wordt een gedeelte van de data gebruikt per epoch om het trainen te versnellen 

In [ ]:


train_ds = tensorflow.keras.preprocessing.image_dataset_from_directory(
  "./humans/train/",
  validation_split = 0.8,
  subset = "training",
  seed = 123,
  image_size = (image_height, image_width),
  batch_size = batch_size)

val_ds = tensorflow.keras.preprocessing.image_dataset_from_directory(
  "./humans/validation/",
  validation_split = 0.8,
  subset = "validation",
  seed = 123,
  image_size = (image_height, image_width),
  batch_size = batch_size)


In [ ]:
history = model.fit(
  train_ds,
  validation_data = val_ds,
  epochs = epochs,
  callbacks = [CustomCallback()],
)
model.save("testen")

In [ ]:
accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']
loss = history.history['loss']
validation_loss = history.history['val_loss']

In [ ]:
epochs_range = range(epochs)

In [ ]:
plt.figure(figsize = (8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label = 'Training accuracy')
plt.plot(epochs_range, validation_accuracy, label = 'Validation accuracy')
plt.legend(loc = 'lower right')
plt.title('Training en validation accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, validation_loss, label = 'Validation loss')
plt.legend(loc = 'upper right')
plt.title('Training en validation Loss')
plt.show()